David Flowers II \
Z1942130 \
Assignment 3 \
Koop FA2024

In [1]:
class Academic:
    MAX_CREDITS = 16
    def __init__(self, campus_id, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
        self.campus_id = campus_id
        self.max_credits = self.MAX_CREDITS
        self.schedule = Schedule()

    # Adds course if it fits within credit bounds
    def add_course(self, course):
        if (self.schedule.credits + course.credits) > self.max_credits:
            raise ValueError(f"Cannot add course {course.course_name}, max credits exceeded.")
        self.schedule.add_course(course)

    def remove_course(self, course):
        self.schedule.remove_course(course)

class Student(Academic):
    # These don't need extra memory for every instance
    allowed_levels = [ "Freshman", "Sophomore", "Junior", "Senior", "Graduate" ]
    def __init__(self, id, first, last, level):
        super().__init__(id, first, last)
        self._level = level

    # _level to prevent recursion issues
    @property
    def level(self):
        return self._level

    # Level must be one of those defined
    @level.setter
    def level(self, level):
        if level not in self.allowed_levels:
            raise ValueError(f"Level must be one of: {allowed_levels}")
        else:
            return _level

    # Add and drop courses from Academic class
    def add_course(self, course):
        super().add_course(course)

    def remove_course(self, course):
        super().remove_course(course)

    # Adds support for + and -
    def __add__(self, course):
        self.add_course(course)

    def __sub__(self, course):
        self.remove_course(course)

# Easily my favorite implementation
class GraduateStudent(Student):
    def __init__(self, id, first, last):
        super().__init__(id, first, last, "Graduate")
        self.max_credits = 12

class Instructor(Academic):
    # These don't need extra memory for every instance
    allowed_ranks = [ "Assistant Professor", "Associate Professor", "Professor", "Instructor" ]
    def __init__(self, id, first, last, rank):
        super().__init__(id, first, last)
        self.max_credits = 9
        self._rank = rank

    # _rank to prevent recursion issues
    @property
    def rank(self):
        return self._rank

    # Rank must be one of those defined
    @rank.setter
    def rank(self, rank):
        if rank not in self.allowed_ranks:
            raise ValueError(f"Level must be one of: {allowed_ranks}")
        else:
            return rank

    # Add/Remove course to super class
    # and assign/unassign self as the instructor
    def add_course(self, course):
        super().add_course(course)
        course.instructor = self

    def remove_course(self, course):
        super().remove_course(course)
        course.instructor = None

    # Adds support for + and -
    def __add__(self, course):
        self.add_course(course)

    def __sub__(self, course):
        self.remove_course(course)

class Course:
    def __init__(self, department, course_num, course_name, section, credits, times, instructor=None):
        self.department = department
        self.course_num = course_num
        self.course_name = course_name
        self.section = section
        self.credits = credits
        self._times = times
        self.instructor = instructor
        self.enrolled_students = []

    # _times to prevent recursion issues
    @property
    def times(self):
        return self._times

    # Times must be a list of tuple
    # (according to the given testing code)
    @times.setter
    def times(self, times):
        if isinstance(value, list) and all(isinstance(item, tuple) for item in value):
            return times
        else:
            raise TypeError("Course times must be tuple.")

    # Prints course in format:
    # ABCD-0000 Course Name (Section 0) (0 credit hours)
    # On Day from 0 to 23 and Day from 0 to 23 and Day from 0 to 23 
    # Taught by Instructor First Last
    def __str__(self):
        printable = ""
        tmptime = self.times
        if self.instructor:
            tmp_inst = f"{self.instructor.rank} {self.instructor.first_name} {self.instructor.last_name}"
        else:
            tmp_inst = "TBD"
        printable = f"{self.department}-"
        printable += f"{self.course_num} "
        printable += f"{self.course_name} "
        printable += f"(Section {self.section}) "
        printable += f"({self.credits} credit hours)\nOn "

        number_of_days = len(tmptime) - 1
        for index, day in enumerate(tmptime):
            printable += f"{day[0]} from {day[1]} to {day[2]} "
            if number_of_days != index:
                printable += "and "
        
        printable += f"\nTaught by {tmp_inst}"

        return printable + "\n"

    # Must be academic or subclass
    def enroll(self, student):
        if isinstance(student, Academic):
            self.enrolled_students.append(student)
        else:
            raise ValueError("Must be Academic or subclass.")

    def drop(self, student):
        for std in self.enrolled_students:
            if student.campus_id == std.campus_id:
                self.enrolled_students.remove(std)

class Schedule:
    def __init__(self, courses=None):
        self.course_list = [] if courses is None else list(courses)

    @property
    def credits(self):
        return sum(course.credits for course in self.course_list)

    # Add course to schedule
    def add_course(self, course):
        self.course_list.append(course)

    def remove_course(self, course):
        self.course_list.remove(course)

    # Support for + and -
    def __add__(self, course):
        self.course_list.append(course)

    def __sub__(self, course):
        self.course_list.remove(course)

    # Support for printing schedules
    # Prints every course in schedule, and their 
    # current credit hours at the bottom
    def __str__(self):
        temp = ""
        for c in self.course_list:
            temp += str(c) + "\n"
        return temp + f"Total Credit Hours: {self.credits}\n"

class Registrar:
    def __init__(self):
        self.all_academics = {}
        self.all_courses = {}

    # Must be Academic or subclass to add
    def add_persons(self, persons):
        for person in persons:
            if isinstance(person, Academic):
                self.all_academics[person.campus_id] = person
            else:
                raise ValueError("All items in persons list must be Academic class (or sub)")

    # Must be Course to add
    def add_courses(self, courses):
        for course in courses:
            if isinstance(course, Course):
                course_key = (course.department, course.course_num, course.section)
                self.all_courses[course_key] = course
            else:
                raise ValueError("All items in courses list must be Course class")
                
    # These functions could easily be merged into a helper function,
    # but I am going to follow the outline on the assignment
    def add_person_to_course(self, campus_id, department, course_num, section):
        # Get required information about course and person
        person = self.all_academics.get(campus_id)
        course_key = (department, course_num, section)
        course = self.all_courses.get(course_key)

        # Check if either course or person does not exist
        if not person:
            raise ValueError("Person could not be found")
        if not course:
            raise ValueError("Course could not be found")

        # Execute
        person.add_course(course)
        course.enroll(person)

    def remove_person_from_course(self, campus_id, department, course_num, section):
        # Get required information about course and person
        person = self.all_academics.get(campus_id)
        course_key = (department, course_num, section)
        course = self.all_courses.get(course_key)

        # Check if either course or person does not exist
        if not person:
            raise ValueError("Person could not be found")
        if not course:
            raise ValueError("Course could not be found")

        # Execute
        person.remove_course(course)
        course.drop(person)

s1 = Student("z143", "Catherine", "Smith", "Senior")
s2 = Student("z352", "Niraj", "Kumar", "Sophomore")
s3 = GraduateStudent("z785", "Divya", "Bharti")
s4 = GraduateStudent("z982", "James", "O'Brien")

i1 = Instructor("a421", "Jennifer", "Martinez", "Professor")
i2 = Instructor("a572", "Jonathan", "Jones", "Instructor")

c1 = Course("CSCI", 1543, "Programming Principles in Python", 1, 3, [("Mon", 10, 12), ("Wed", 10, 12)])
c2 = Course("CSCI", 1342, "Computer Networks", 2, 4, [("Tue", 14, 16), ("Thu", 14, 16), ("Fri", 12, 13)])
c3 = Course("CSCI", 1352, "Computer Graphics", 1, 3, [("Tue", 10, 12), ("Thu", 10, 12)])
c4 = Course("SOCI", 1230, "Introduction to Sociology", 1, 3, [("Mon", 11, 13), ("Thu", 11, 13)])
c5 = Course("POLS", 1100, "American Politics", 2, 3, [("Tue", 10, 12), ("Thu", 10, 12)])
c6 = Course("SOCI", 1450, "Classical Sociological Theory", 1, 3, [("Mon", 12, 13), ("Wed", 12, 13), ("Fri", 12, 13)])

r = Registrar()
r.add_persons([s1,s2,s3,s4,i1,i2])
r.add_courses([c1,c2,c3,c4,c5, c6])

r.add_person_to_course("a572", "SOCI", 1230, 1)
r.add_person_to_course("a572", "POLS", 1100, 2) # error due to conflict (490, doesn't check)

#r.change_course_time("SOCI", 1230, 1, [("Mon", 9, 11), ("Wed", 9, 11)])
r.add_person_to_course("a572", "SOCI", 1450, 1) # (no change_time method)
r.add_person_to_course("a421", "CSCI", 1543, 1)
r.add_person_to_course("a421", "CSCI", 1342, 2)
#r.add_person_to_course("a421", "CSCI", 1352, 1) # error due to max credits

r.add_person_to_course("z785", "SOCI", 1230, 1)
r.add_person_to_course("z785", "CSCI", 1352, 1) # (no change_time method)
r.add_person_to_course("z143", "CSCI", 1543, 1)
r.add_person_to_course("z143", "CSCI", 1342, 2)
r.add_person_to_course("z143", "SOCI", 1230, 1) # error due to conflict (490, doesn't check)

#r.change_course_time( "SOCI", 1230, 1, [("Tue", 11, 13), ("Thu", 11, 13)]) # error due to conflict
#r.change_course_time( "SOCI", 1230, 1, [("Tue", 12, 14), ("Thu", 12, 14)])
r.add_person_to_course("z143", "SOCI", 1230, 1) # error due to conflict (490, doesn't check)

for s in [s1,s2,s3,s4,i1,i2]:
    # This header is for easily discerning student schedules
    print(40*"-")
    print(f"Schedule for {s.first_name} {s.last_name} ({s.campus_id})")
    print(40*"-")
    print(s.schedule)

----------------------------------------
Schedule for Catherine Smith (z143)
----------------------------------------
CSCI-1543 Programming Principles in Python (Section 1) (3 credit hours)
On Mon from 10 to 12 and Wed from 10 to 12 
Taught by Professor Jennifer Martinez

CSCI-1342 Computer Networks (Section 2) (4 credit hours)
On Tue from 14 to 16 and Thu from 14 to 16 and Fri from 12 to 13 
Taught by Professor Jennifer Martinez

SOCI-1230 Introduction to Sociology (Section 1) (3 credit hours)
On Mon from 11 to 13 and Thu from 11 to 13 
Taught by Instructor Jonathan Jones

SOCI-1230 Introduction to Sociology (Section 1) (3 credit hours)
On Mon from 11 to 13 and Thu from 11 to 13 
Taught by Instructor Jonathan Jones

Total Credit Hours: 13

----------------------------------------
Schedule for Niraj Kumar (z352)
----------------------------------------
Total Credit Hours: 0

----------------------------------------
Schedule for Divya Bharti (z785)
--------------------------------------